In [1]:
import nltk
import numpy as np
import json
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score as meteor

import os
from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /home/guest/thdaryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# !ls /scratch2/thdaryan/data/fine_tuned/refCOCO/test/labels

In [3]:
labels_folder_path = '/scratch2/thdaryan/data/fine_tuned/refCOCO/test/labels'
result_folder_path = 'result'

In [4]:
with open(os.path.join(labels_folder_path, 'lab_0.json')) as f:
    sample_label = json.load(f)
print(sample_label['ref_sents'])

ref = sample_label['ref_sents']
exp = [word_tokenize(r.lower()) for r in ref]
print(exp)

['guy petting elephant', 'foremost person', 'green shirt']
[['guy', 'petting', 'elephant'], ['foremost', 'person'], ['green', 'shirt']]


In [5]:
print(sentence_bleu(exp, ['foremost'], weights=(1,0,0,0)))

0.36787944117144233


/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoothi

In [6]:
with open(os.path.join(result_folder_path, 'result_pragmatic_caption_0_1000.json')) as f:
    result_0_1000 = json.load(f)
print(result_0_1000['0'])
print(result_0_1000['0'][0][1:-1])

['^man is sitting on the ground$', '-5.629921482260508']
man is sitting on the ground


## Calculate Bleu Score

In [7]:
bleu_scores = []
for file in tqdm(range(0, 5000, 1000)):
    print(file, file+1000)
    with open(os.path.join(result_folder_path, f'result_pragmatic_caption_{file}_{file+1000}.json')) as f:
        list_result = json.load(f)
    for idx in list_result:
        result = list_result[idx]
        if (result[0]=='ERROR'):
#             print(idx, result)
            continue

        with open(os.path.join(labels_folder_path, f'lab_{idx}.json')) as f:
            sample_label = json.load(f)

        ref = sample_label['ref_sents']
        ref = [word_tokenize(r.lower()) for r in ref]

        generated_exp = result[0][1:-1]
        score = sentence_bleu(ref, word_tokenize(generated_exp), weights=(1,0,0,0))

#         print(score, ref,  word_tokenize(generated_exp))
        bleu_scores.append(score)

  0%|          | 0/5 [00:00<?, ?it/s]/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider us

0 1000


 20%|██        | 1/5 [00:00<00:01,  2.72it/s]

1000 2000


 40%|████      | 2/5 [00:00<00:01,  2.73it/s]

...

 60%|██████    | 3/5 [00:01<00:00,  2.74it/s]

3000 4000


 80%|████████  | 4/5 [00:01<00:00,  2.75it/s]

4000 5000


100%|██████████| 5/5 [00:01<00:00,  2.76it/s]


In [8]:
len(bleu_scores)

4984

In [9]:
np.average(bleu_scores)

0.168943800017137

## Calculate Rouge Score

In [10]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [14]:
rouge1_scores = []
rougeL_scores = []
for file in tqdm(range(0, 5000, 1000)):
    print(file, file+1000)
    with open(os.path.join(result_folder_path, f'result_pragmatic_caption_{file}_{file+1000}.json')) as f:
        list_result = json.load(f)
    for idx in list_result:
        result = list_result[idx]
        if (result[0]=='ERROR'):
#             print(idx, result)
            continue

        with open(os.path.join(labels_folder_path, f'lab_{idx}.json')) as f:
            sample_label = json.load(f)

        ref = sample_label['ref_sents']
        generated_exp = result[0][1:-1]
        rouge1_scores.append(np.average([scorer.score(target, generated_exp)['rouge1'].recall for target in ref]))
        rougeL_scores.append(np.average([scorer.score(target, generated_exp)['rougeL'].recall for target in ref]))

  0%|          | 0/5 [00:00<?, ?it/s]

0 1000


 20%|██        | 1/5 [00:00<00:02,  1.33it/s]

1000 2000


 40%|████      | 2/5 [00:01<00:02,  1.34it/s]

...

 60%|██████    | 3/5 [00:02<00:01,  1.34it/s]

3000 4000


 80%|████████  | 4/5 [00:02<00:00,  1.35it/s]

4000 5000


100%|██████████| 5/5 [00:03<00:00,  1.36it/s]


In [15]:
print(np.average(rouge1_scores), np.std(rouge1_scores))

0.14807628272710655 0.17988326157351756


In [16]:
print(np.average(rougeL_scores), np.std(rougeL_scores))

0.14305185991986696 0.17403894462041697


## Calculate Meteor Score

In [21]:
meteor_scores = []
for file in tqdm(range(0, 5000, 1000)):
    print(file, file+1000)
    with open(os.path.join(result_folder_path, f'result_pragmatic_caption_{file}_{file+1000}.json')) as f:
        list_result = json.load(f)
    for idx in list_result:
        result = list_result[idx]
        if (result[0]=='ERROR'):
#             print(idx, result)
            continue

        with open(os.path.join(labels_folder_path, f'lab_{idx}.json')) as f:
            sample_label = json.load(f)

        ref = sample_label['ref_sents']
        ref = [r.lower() for r in ref]

        generated_exp = result[0][1:-1]
        score = meteor(ref, generated_exp)

        meteor_scores.append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

0 1000


 20%|██        | 1/5 [00:02<00:08,  2.18s/it]

1000 2000


 40%|████      | 2/5 [00:02<00:05,  1.73s/it]

...

 60%|██████    | 3/5 [00:03<00:02,  1.40s/it]

3000 4000


 80%|████████  | 4/5 [00:04<00:01,  1.16s/it]

4000 5000


100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


In [23]:
print(np.average(meteor_scores), np.std(meteor_scores))

0.13534073040601782 0.15401653259086828
